<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=edfc29f4c641f616d7ef2bf9d0a36bff2de51358d3e17ef2e75a967d50922ae3
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-10 12:25:47
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.23 C
CY Investment:  1.50 C
Reserve:  57.44 K
Current:  1.42 C
-------------------
Today PnL: 54.60 K (0.39%)
Current PnL: -19.86 L (-13.23%)
CY Booked + Current PnL: -8.42 L (-5.61%)
-------------------
Total profit:  1.83 L
Total loss:  -21.68 L
-------------------
Total Booked + Current PnL: 18.56 L (15.08%)
Total Booked PnL: 38.42 L (31.2%)
Curr Year Booked PnL: 11.44 L (8.07%)
Prev Year Booked PnL: 26.98 L (21.91%)
Est FTT:  2.29 C
Est FTT PnL: 87.36 L (61.64%)
Deployed:  1.23 C
Current:  1.42 C
CAGR/XIRR %: 8.47%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-40.35,64.0,H-MC,14.26,118678.0,18174.0,10930.0,-2.45,18.08,9.21,28.96,79.0,1.66,0.84,58.19,MH,ATH,METALS
14,BLUESTARCO,2080.00,12.39,57.0,X-MC,3.20,193950.0,29280.0,14042.0,2.49,17.78,7.24,26.31,89.0,2.09,1.37,27.18,X40N,NTT,AC
19,CIPLA,1662.74,-20.12,57.0,X-LC,3.67,213062.0,8562.0,14723.0,2.78,4.19,6.91,11.39,10.0,0.58,1.51,13.51,X40N,ATH,PHARMA
84,VOLTAS,1530.00,2.28,64.0,X-MC,1.12,214680.0,22938.0,14813.0,3.56,11.96,6.90,19.69,99.0,1.55,1.52,19.57,XY25,NTT,AC
77,TTKPRESTIG,770.00,93.17,39.0,M-SC,1.00,83270.0,-17507.0,17603.0,-0.22,-17.37,21.14,0.09,245.0,-0.99,0.59,9.40,OX40N,NTT,DURABLES


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,VOLTAS,1530.00,2.28,64.0,X-MC,1.12,214680.0,22938.0,14813.0,3.56,11.96,6.90,19.69,99.0,1.55,1.52,19.57,XY25,NTT,AC
21,COFFEEDAY,80.00,-43.66,56.0,L-SC,29.30,84461.0,-29088.0,69140.0,3.02,-25.62,81.86,35.27,268.0,-0.42,0.60,106.72,XR,NTT,HOTELS
46,KPIGREEN,731.05,-8.25,36.0,H-SC,6.28,109456.0,-15841.0,74769.0,2.98,-12.64,68.31,47.03,141.0,-0.21,0.78,37.05,MH,ATH,POWER
19,CIPLA,1662.74,-20.12,57.0,X-LC,3.67,213062.0,8562.0,14723.0,2.78,4.19,6.91,11.39,10.0,0.58,1.51,13.51,X40N,ATH,PHARMA
53,QUESS,986.00,-46.01,36.0,X-SC,37.76,53567.0,-11439.0,162367.0,2.77,-17.60,303.11,232.18,198.0,-0.07,0.38,2.77,XY24,NTT,MISC


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,HINDZINC,730.22,31.71,62.0,M-LC,7.74,214651.0,9575.0,102990.0,-3.67,4.67,47.98,54.89,52.0,0.09,1.52,29.89,X5K,ATH,METALS
60,SAIL,228.00,44.35,51.0,M-MC,12.68,228372.0,3410.0,164245.0,-2.81,1.52,71.92,74.53,192.0,0.02,1.62,34.26,XY24,BTT,STEEL
72,TATAELXSI,9161.00,-23.88,46.0,H-MC,6.63,103008.0,-28827.0,71055.0,-2.72,-21.87,68.98,32.03,98.0,-0.41,0.73,15.38,OX40N,NTT,IT
50,NATIONALUM,244.55,-40.35,64.0,H-MC,14.26,118678.0,18174.0,10930.0,-2.45,18.08,9.21,28.96,79.0,1.66,0.84,58.19,MH,ATH,METALS
26,GREENPANEL,537.00,227.52,55.0,M-SC,4.62,146254.0,-36824.0,115804.0,-1.75,-20.11,79.18,43.14,230.0,-0.32,1.04,37.57,XY24,NTT,MISC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,107.70,63.0,H-SC,13.77,131166.0,-11313.0,30837.0,0.77,-7.94,23.51,13.70,163.0,-0.37,0.93,54.81,OX40N,NTT,BANKS
77,TTKPRESTIG,770.0,93.17,39.0,M-SC,1.00,83270.0,-17507.0,17603.0,-0.22,-17.37,21.14,0.09,245.0,-0.99,0.59,9.40,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-67.97,48.0,H-SC,1.92,220905.0,-48762.0,85093.0,0.04,-18.08,38.52,13.47,138.0,-0.57,1.56,12.32,XY24,NTT,PAINTS
72,TATAELXSI,9161.0,-23.88,46.0,H-MC,6.63,103008.0,-28827.0,71055.0,-2.72,-21.87,68.98,32.03,98.0,-0.41,0.73,15.38,OX40N,NTT,IT
18,CERA,9475.0,-24.58,33.0,H-SC,1.81,137310.0,-38593.0,80615.0,-0.88,-21.94,58.71,23.89,149.0,-0.48,0.97,18.59,OX40N,NTT,CERAMICS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.00,44.35,51.0,M-MC,12.68,228372.0,3410.0,164245.0,-2.81,1.52,71.92,74.53,192.0,0.02,1.62,34.26,XY24,BTT,STEEL
32,HINDZINC,730.22,31.71,62.0,M-LC,7.74,214651.0,9575.0,102990.0,-3.67,4.67,47.98,54.89,52.0,0.09,1.52,29.89,X5K,ATH,METALS
79,UNIONBANK,163.00,-11.63,57.0,M-LC,9.19,158521.0,17681.0,26806.0,1.15,12.55,16.91,31.59,66.0,0.66,1.12,42.37,XY24,NTT,BANKS
86,WIPRO,420.00,-11.98,57.0,M-LC,5.69,154857.0,3912.0,105535.0,1.37,2.59,68.15,72.51,53.0,0.04,1.10,8.77,XR,NTT,IT
11,BANKINDIA,190.00,-24.22,69.0,H-MC,11.96,195480.0,15672.0,96743.0,1.04,8.72,49.49,62.52,88.0,0.16,1.38,45.77,XR,NTT,BANKS


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-40.35,64.0,H-MC,14.26,118678.0,18174.0,10930.0,-2.45,18.08,9.21,28.96,79.0,1.66,0.84,58.19,MH,ATH,METALS
11,BANKINDIA,190.00,-24.22,69.0,H-MC,11.96,195480.0,15672.0,96743.0,1.04,8.72,49.49,62.52,88.0,0.16,1.38,45.77,XR,NTT,BANKS
32,HINDZINC,730.22,31.71,62.0,M-LC,7.74,214651.0,9575.0,102990.0,-3.67,4.67,47.98,54.89,52.0,0.09,1.52,29.89,X5K,ATH,METALS
86,WIPRO,420.00,-11.98,57.0,M-LC,5.69,154857.0,3912.0,105535.0,1.37,2.59,68.15,72.51,53.0,0.04,1.10,8.77,XR,NTT,IT
37,INDIAMART,4810.62,-54.64,42.0,H-SC,8.73,126670.0,3334.0,128291.0,-0.92,2.70,101.28,106.72,119.0,0.03,0.90,26.71,AR,ATH,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,ASIANPAINT,3465.66,-18.16,35.0,X-LC,5.55,206483.0,-45285.0,98492.0,0.43,-17.99,47.70,21.13,27.0,-0.46,1.46,11.07,X40,ATH,PAINTS
23,DABUR,735.00,-12.12,36.0,X-MC,3.76,237796.0,-10272.0,118684.0,1.01,-4.14,49.91,43.70,102.0,-0.09,1.68,9.71,XY24,BTT,FMCG
53,QUESS,986.00,-46.01,36.0,X-SC,37.76,53567.0,-11439.0,162367.0,2.77,-17.60,303.11,232.18,198.0,-0.07,0.38,2.77,XY24,NTT,MISC
22,COLPAL,3767.14,-4.76,38.0,X-MC,7.27,222190.0,-41175.0,154533.0,0.60,-15.63,69.55,43.04,84.0,-0.27,1.57,3.15,XY25,ATH,FMCG
56,RELAXO,1176.00,-39.88,38.0,X-SC,2.54,81949.0,-63211.0,139141.0,-0.46,-43.55,169.79,52.31,136.0,-0.45,0.58,9.69,X40N,NTT,FOOTWEAR


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
29,HAVELLS,2069.16,-4.48,47.0,X-MC,0.50,236746.0,-10985.0,86033.0,2.14,-4.43,36.34,30.30,92.0,-0.13,1.68,8.27,X40,ATH,ELECTRICAL
84,VOLTAS,1530.00,2.28,64.0,X-MC,1.12,214680.0,22938.0,14813.0,3.56,11.96,6.90,19.69,99.0,1.55,1.52,19.57,XY25,NTT,AC
73,TATAMOTORS,1065.00,-22.16,43.0,X-LC,1.58,245860.0,-30663.0,140730.0,-0.56,-11.09,57.24,39.81,54.0,-0.22,1.74,17.83,XY24,NTT,AUTO
65,SIEMENS,4671.50,-1.54,48.0,H-LC,1.67,158100.0,-27995.0,75477.0,-0.52,-15.04,47.74,25.51,15.0,-0.37,1.12,16.51,AR,ATH,ELECTRICAL
34,ICICIGI,2260.25,-18.95,49.0,X-MC,1.86,139527.0,3394.0,27738.0,0.15,2.49,19.88,22.86,91.0,0.12,0.99,15.42,X40,ATH,INSURANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-46.01,36.0,X-SC,37.76,53567.0,-11439.0,162367.0,2.77,-17.60,303.11,232.18,198.0,-0.07,0.38,2.77,XY24,NTT,MISC
22,COLPAL,3767.14,-4.76,38.0,X-MC,7.27,222190.0,-41175.0,154533.0,0.60,-15.63,69.55,43.04,84.0,-0.27,1.57,3.15,XY25,ATH,FMCG
83,VBL,671.64,-20.49,43.0,X-LC,10.31,286558.0,-29284.0,141961.0,0.73,-9.27,49.54,35.67,5.0,-0.21,2.03,3.30,X40N,ATH,FMCG
42,ITC,452.00,-38.77,46.0,X-LC,2.40,197091.0,-3047.0,23927.0,0.79,-1.52,12.14,10.44,4.0,-0.13,1.40,3.96,X40,NTT,FMCG
9,BAJAJHFL,181.50,-14.27,39.0,X-MC,4.95,180597.0,-19909.0,117063.0,0.03,-9.93,64.82,48.45,90.0,-0.17,1.28,4.14,X40N,ATH,FINANCE


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-46.01,36.0,X-SC,37.76,53567.0,-11439.0,162367.0,2.77,-17.60,303.11,232.18,198.0,-0.07,0.38,2.77,XY24,NTT,MISC
56,RELAXO,1176.00,-39.88,38.0,X-SC,2.54,81949.0,-63211.0,139141.0,-0.46,-43.55,169.79,52.31,136.0,-0.45,0.58,9.69,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,-34.87,43.0,X-SC,4.42,97309.0,-31361.0,76660.0,0.01,-24.37,78.78,35.20,219.0,-0.41,0.69,11.69,X40,NTT,FOOTWEAR
17,CAMS,5211.76,-4.94,51.0,X-SC,1.45,108136.0,6132.0,37794.0,0.81,6.01,34.95,43.06,122.0,0.16,0.77,24.16,X40N,ATH,MISC
33,HONAUT,58357.33,-22.51,48.0,X-SC,2.14,109125.0,-18813.0,65944.0,0.37,-14.71,60.43,36.84,143.0,-0.29,0.77,11.72,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4406.34,-28.69,49.0,X-LC,12.18,283692.0,-62264.0,130498.0,-1.43,-18.00,46.00,19.73,1.0,-0.48,2.01,4.49,X40,ATH,IT
40,INFY,2275.00,-17.44,57.0,X-LC,7.78,322312.0,9424.0,162252.0,0.26,3.01,50.34,54.87,3.0,0.06,2.28,9.86,X40,BTT,IT
42,ITC,452.00,-38.77,46.0,X-LC,2.40,197091.0,-3047.0,23927.0,0.79,-1.52,12.14,10.44,4.0,-0.13,1.40,3.96,X40,NTT,FMCG
83,VBL,671.64,-20.49,43.0,X-LC,10.31,286558.0,-29284.0,141961.0,0.73,-9.27,49.54,35.67,5.0,-0.21,2.03,3.30,X40N,ATH,FMCG
1,ABB,7934.00,-40.45,46.0,H-LC,7.50,248784.0,-12835.0,132055.0,0.71,-4.91,53.08,45.57,7.0,-0.10,1.76,5.86,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-43.66,56.0,L-SC,29.30,84461.0,-29088.0,69140.0,3.02,-25.62,81.86,35.27,268.0,-0.42,0.60,106.72,XR,NTT,HOTELS
49,MASFIN,398.61,-20.05,42.0,H-SC,10.37,90675.0,-7305.0,28907.0,0.35,-7.46,31.88,22.05,152.0,-0.25,0.64,31.80,XR,ATH,FINANCE
50,NATIONALUM,244.55,-40.35,64.0,H-MC,14.26,118678.0,18174.0,10930.0,-2.45,18.08,9.21,28.96,79.0,1.66,0.84,58.19,MH,ATH,METALS
78,UJJIVANSFB,60.00,107.70,63.0,H-SC,13.77,131166.0,-11313.0,30837.0,0.77,-7.94,23.51,13.70,163.0,-0.37,0.93,54.81,OX40N,NTT,BANKS
26,GREENPANEL,537.00,227.52,55.0,M-SC,4.62,146254.0,-36824.0,115804.0,-1.75,-20.11,79.18,43.14,230.0,-0.32,1.04,37.57,XY24,NTT,MISC


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAMMAANCAP,326.00,-185.23,67.0,M-SC,15.45,147564.0,-1656.0,145838.0,1.80,-1.11,98.83,96.62,208.0,-0.01,1.05,59.70,XY25,NTT,FINANCE
69,SURYODAY,240.00,68.86,67.0,H-SC,9.09,154846.0,-24225.0,89950.0,2.16,-13.53,58.09,36.71,135.0,-0.27,1.10,53.64,XR,NTT,BANKS
32,HINDZINC,730.22,31.71,62.0,M-LC,7.74,214651.0,9575.0,102990.0,-3.67,4.67,47.98,54.89,52.0,0.09,1.52,29.89,X5K,ATH,METALS
26,GREENPANEL,537.00,227.52,55.0,M-SC,4.62,146254.0,-36824.0,115804.0,-1.75,-20.11,79.18,43.14,230.0,-0.32,1.04,37.57,XY24,NTT,MISC
68,STARHEALTH,761.00,16.85,68.0,H-SC,7.41,260250.0,-20873.0,153730.0,-0.04,-7.42,59.07,47.26,144.0,-0.14,1.84,39.43,XY24,NTT,INSURANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.14
1,25,43.43
2,50,75.18


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.96
LC    32.30
MC    23.76
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.85
X40      14.64
X40N     11.29
XR       11.00
XY25     10.45
AR        8.17
OX40N     7.62
X200      1.75
MH        1.62
X5K       1.52
SR        1.11
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    25.46
X-LC    22.92
X-MC    17.23
M-SC    12.51
H-LC     4.63
H-MC     4.60
X-SC     4.53
M-LC     3.74
M-MC     1.62
L-SC     1.46
L-LC     1.01
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.76,-5.65,40.47
IT,12.86,-15.89,78.66
FINANCE,10.51,-13.70,68.61
MISC,6.89,-18.22,83.73
BANKS,6.42,-10.16,68.10
PAINTS,5.47,-19.48,37.29
ELECTRICAL,5.33,-9.38,47.76
INSURANCE,3.80,-3.17,41.74
AC,3.58,4.24,11.79


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3217220.0,22
XR,1359271.0,14
AR,1176868.0,9
X40,775059.0,10
X40N,641420.0,9
OX40N,571710.0,10
XY25,462779.0,7
SR,264873.0,2
X5K,102990.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3405740.0,25
M-SC,1278124.0,15
X-MC,1102753.0,12
X-LC,1006371.0,13
X-SC,541324.0,6
H-MC,359381.0,4
H-LC,286107.0,3
L-SC,255918.0,3
M-LC,235331.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1196481.0      6
           AR         805990.0      5
           XR         798587.0      7
M-SC       XY24       690212.0      6
X-MC       XY24       589247.0      4
X-LC       X40        485344.0      5
X-SC       X40N       302297.0      4
H-SC       OX40N      265040.0      4
           SR         264873.0      2
X-MC       X40        213055.0      4
X-LC       X40N       208018.0      3
H-LC       AR         207532.0      2
X-LC       XY24       207209.0      2
H-MC       XY24       180653.0      1
X-MC       XY25       169346.0      2
L-SC       XR         167740.0      2
M-MC       XY24       164245.0      1
M-SC       AR         163346.0      2
X-SC       XY24       162367.0      1
M-SC       OX40N      147437.0      4
           XY25       145838.0      1
           XR         131291.0      2
X-MC       X40N       131105.0      2
X-LC       XY25       105800.0      3
M-LC       XR         105535.0      1
           X5K        102990.0      1
H-MC       XR          96743.0      1
L-SC       OX40N       88178.0      1
H-LC       X200        78575.0      1
X-SC       X40         76660.0      1
H-SC       MH          74769.0      1
H-MC       OX40N       71055.0      1
L-MC       XR          59375.0      1
L-LC       XY25        41795.0      1
M-LC       XY24        26806.0      1
H-MC       MH          10930.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 26.0 seconds
